# Pandas 데모 - Explarotary Data Analysis

## 라이브러리 import 및 설정

In [47]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [48]:
import lightgbm as lgb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import warnings
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV

In [49]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import StandardScaler, RobustScaler # 데이터 표준화

# lightgbm 모델
from lightgbm import LGBMClassifier, plot_importance

# normalization
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

#feature Selection
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif
import itertools

In [50]:
rcParams['figure.figsize'] = (18, 9)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

## 데이터 다운로드

데이터는 [Dacon 단국대 소/중 데이터 분석 AI 경진대회 웹사이트](https://www.dacon.io/competitions/official/235638/data/)에서 다운로드 받아 `../input` 폴더에 저장.

In [51]:
!ls -alF ../input/

total 4408604
drwxr-xr-x  2 swcu swcu       4096  9월 29 19:09 ./
drwxrwxr-x 10 swcu swcu       4096 10월  5 21:59 ../
-rw-r--r--  1 swcu swcu   47899736  9월 18 02:22 ecminer_standard_normal_outlier_train.csv
-rw-r--r--  1 swcu swcu  132585197  9월 24 15:52 feature.csv
-rw-r--r--  1 swcu swcu     720009  9월 23 11:34 lgbm_u_r_i_z_redshift_du_dg_dr_di_dz.csv
-rw-r--r--  1 swcu swcu     800010  7월 26 13:35 sample_submission.csv
-rw-r--r--  1 swcu swcu   17619545  9월 18 01:17 standard_normal_test.xlsx
-rw-r--r--  1 swcu swcu   18936246  7월 26 13:35 test.csv
-rw-r--r--  1 swcu swcu   76278443  7월 26 13:35 train.csv
-rw-r--r--  1 swcu swcu 3360982474  9월 30 10:56 trn_engineering.csv
-rw-r--r--  1 swcu swcu  858552178  9월 30 10:57 tst_engineering.csv


In [52]:
data_dir = Path('../input/')
trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
# feature_file = data_dir / 'feature.csv'
sample_file = data_dir / 'sample_submission.csv'
submission_dir = Path('../submission')
target_col = 'class'
seed = 42

In [53]:
algo_name = 'lgb'
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

#feature_file = feature_dir / f'{feature_name}.csv'
submission_file = submission_dir / f'{model_name}.csv'

## EDA

In [54]:
trn = pd.read_csv(trn_file, index_col=0)
print(trn.shape)
trn.head()

(320000, 19)


,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
id,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,13,12,1.1939,1.1943,1.1937,1.1938,1.1941,0


## 데이터 개요

In [55]:
trn.describe()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
count,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,3.2000e+05,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000
mean,19.8524,18.3768,17.7358,17.1742,16.8752,5.8308e-02,19.6289,18.2271,17.6154,17.1523,16.8693,6.3327,6.1388,1.1758,1.1765,1.1751,1.1754,1.1761,1.1163
std,1.9455,31.1108,1.4662,52.5615,57.1463,5.7546e-01,1.9236,17.7866,1.4730,25.0758,35.4365,8.8811,8.5516,0.1163,0.1181,0.1147,0.1155,0.1172,0.9235
min,-17.6875,-17555.3656,7.7314,-23955.8178,-23955.8178,-1.6605e+02,-42.8375,-9999.0000,-16.3802,-9999.0000,-9999.0000,1.0000,1.0000,1.0001,1.0000,1.0002,1.0001,1.0000,0.0000
25%,18.7242,17.4749,16.8772,16.5239,16.2889,3.5580e-05,18.5636,17.3484,16.7871,16.4530,16.2341,1.0000,1.0000,1.0882,1.0884,1.0878,1.0880,1.0883,0.0000
50%,19.4194,18.1405,17.5258,17.1497,16.9176,4.7129e-02,19.2646,18.0224,17.4434,17.0874,16.8694,2.0000,2.0000,1.1793,1.1792,1.1794,1.1794,1.1793,1.0000
75%,20.4321,19.0732,18.4281,18.0074,17.7289,9.4597e-02,20.1976,18.8884,18.2911,17.9070,17.6557,5.0000,5.0000,1.2275,1.2260,1.2292,1.2286,1.2268,2.0000
max,58.6233,46.3383,72.0097,50.1641,52.6127,6.2323e+01,30.7779,30.6132,31.9572,32.0162,30.7813,44.0000,42.0000,2.0508,2.0803,2.0221,2.0363,2.0654,2.0000


In [56]:
trn.dtypes

u            float64
g            float64
r            float64
i            float64
z            float64
redshift     float64
dered_u      float64
dered_g      float64
dered_r      float64
dered_i      float64
dered_z      float64
nObserve       int64
nDetect        int64
airmass_u    float64
airmass_g    float64
airmass_r    float64
airmass_i    float64
airmass_z    float64
class          int64
dtype: object

In [57]:
for i in trn.columns:
    print(i, len(trn.loc[trn[i]<0]))


u 3
g 4
r 0
i 4
z 9
redshift 70876
dered_u 8
dered_g 9
dered_r 6
dered_i 7
dered_z 8
nObserve 0
nDetect 0
airmass_u 0
airmass_g 0
airmass_r 0
airmass_i 0
airmass_z 0
class 0


In [58]:
col = ['u', 'g', 'r', 'i', 'z', 'dered_u', 'dered_g', 
      'dered_r', 'dered_i', 'dered_z']
trn1 = trn.copy()
for i in col:
    trn1 = trn1[trn1[i] >=0]

trn1

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
id,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,13,12,1.1939,1.1943,1.1937,1.1938,1.1941,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319995,18.3656,17.2409,16.7739,16.4850,16.3404,5.1988e-02,18.1328,17.0406,16.6374,16.3861,16.2874,2,2,1.0930,1.0888,1.0973,1.0951,1.0909,2
319996,18.6856,17.6658,17.1904,16.8169,16.6367,8.7261e-02,18.5207,17.5131,17.0871,16.7436,16.6017,2,2,1.0517,1.0508,1.0526,1.0522,1.0513,1
319997,21.1890,18.4234,17.2956,16.9011,16.6841,-1.0148e-05,20.9890,18.3076,17.2152,16.8403,16.6355,31,30,1.1933,1.1941,1.1925,1.1929,1.1937,0


In [59]:
trn1.describe()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
count,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,3.1998e+05,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000
mean,19.8528,18.4320,17.7358,17.3043,17.0613,5.8309e-02,19.6301,18.2593,17.6161,17.2154,16.9947,6.3328,6.1390,1.1758,1.1765,1.1751,1.1754,1.1761,1.1163
std,1.9428,1.6591,1.4662,1.3198,1.3333,5.7548e-01,1.9072,1.6566,1.4665,1.3249,1.3233,8.8814,8.5519,0.1163,0.1181,0.1147,0.1155,0.1172,0.9235
min,2.2651,1.2383,7.7314,2.7492,5.1281,-1.6605e+02,0.7538,3.0828,6.0403,6.8561,7.2567,1.0000,1.0000,1.0001,1.0000,1.0002,1.0001,1.0000,0.0000
25%,18.7242,17.4749,16.8772,16.5239,16.2890,3.5580e-05,18.5637,17.3484,16.7873,16.4531,16.2342,1.0000,1.0000,1.0882,1.0884,1.0878,1.0880,1.0883,0.0000
50%,19.4195,18.1405,17.5258,17.1498,16.9177,4.7133e-02,19.2646,18.0224,17.4434,17.0874,16.8694,2.0000,2.0000,1.1793,1.1792,1.1794,1.1794,1.1793,1.0000
75%,20.4320,19.0729,18.4281,18.0075,17.7290,9.4600e-02,20.1976,18.8885,18.2912,17.9070,17.6556,5.0000,5.0000,1.2275,1.2260,1.2292,1.2286,1.2268,2.0000
max,58.6233,46.3383,72.0097,50.1641,52.6127,6.2323e+01,30.7779,30.6132,31.9572,32.0162,30.7813,44.0000,42.0000,2.0508,2.0803,2.0221,2.0363,2.0654,2.0000


In [60]:
tst = pd.read_csv(tst_file, index_col = 0)
tst

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z
id,,,,,,,,,,,,,,,,,,
320000,19.1431,18.0172,17.4733,17.1144,16.9263,1.2130e-01,19.0750,17.9550,17.4308,17.0838,16.9087,1,1,1.1563,1.1515,1.1613,1.1588,1.1539
320001,18.7409,17.1705,16.3691,15.9865,15.7307,-6.6907e-05,15.0703,14.3098,14.3901,14.5160,14.6372,2,2,1.0076,1.0074,1.0080,1.0078,1.0075
320002,19.2980,17.8803,17.2221,16.8096,16.6352,1.1001e-01,19.1004,17.7726,17.1460,16.7497,16.5689,1,1,1.3809,1.3905,1.3715,1.3761,1.3856
320003,18.0775,17.1569,16.9041,16.8118,16.7955,2.9697e-05,17.9729,17.0735,16.8470,16.7700,16.7668,1,1,1.1408,1.1408,1.1409,1.1408,1.1408
320004,17.8325,16.7646,16.2135,15.8722,15.6621,6.8471e-02,17.7545,16.6794,16.1563,15.8315,15.6433,2,2,1.1110,1.1077,1.1145,1.1127,1.1093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,19.6734,18.6617,18.2708,18.1225,18.0627,-3.1726e-04,19.3361,18.3933,18.0860,17.9861,17.9670,17,17,1.1858,1.1863,1.1855,1.1857,1.1861
399996,21.5229,20.7283,20.6087,21.0049,20.8458,6.5455e-02,21.3993,20.6399,20.5451,20.9507,20.7827,8,4,1.2850,1.2908,1.2793,1.2821,1.2878
399997,19.3991,18.2411,18.1605,18.1978,18.2018,-8.4180e-04,18.9136,17.8753,17.9047,18.0043,18.0400,1,1,1.1885,1.1886,1.1885,1.1885,1.1885


In [61]:
df = pd.concat([trn1, tst])
df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
id,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0.0
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,13,12,1.1939,1.1943,1.1937,1.1938,1.1941,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,19.6734,18.6617,18.2708,18.1225,18.0627,-3.1726e-04,19.3361,18.3933,18.0860,17.9861,17.9670,17,17,1.1858,1.1863,1.1855,1.1857,1.1861,NaN
399996,21.5229,20.7283,20.6087,21.0049,20.8458,6.5455e-02,21.3993,20.6399,20.5451,20.9507,20.7827,8,4,1.2850,1.2908,1.2793,1.2821,1.2878,NaN
399997,19.3991,18.2411,18.1605,18.1978,18.2018,-8.4180e-04,18.9136,17.8753,17.9047,18.0043,18.0400,1,1,1.1885,1.1886,1.1885,1.1885,1.1885,NaN


# Feature Engineering

In [62]:
target_col = 'class'
feature_col = [x for x in trn1.columns if trn1[x].dtype in [np.int64, np.float64] and x !=target_col]
len(feature_col)

18

In [63]:
oneChar_col = ['u','g','r','i','z']
dered_col = [c for c in df.columns if c.find('dered') != -1]
airmass_col = [c for c in df.columns if c.find('airmass') != -1]
print(dered_col)
print(airmass_col)

['dered_u', 'dered_g', 'dered_r', 'dered_i', 'dered_z']
['airmass_u', 'airmass_g', 'airmass_r', 'airmass_i', 'airmass_z']


In [64]:
#모든 조합 차이 feature
diff_feature = []
for c1, c2 in itertools.combinations(feature_col,2):
    new_c = f'{c1}_{c2}_diff'
    df[new_c] = df[c1] - df[c2]
    diff_feature.append(new_c)
# 모든 조합 나누기    
divide_feature = []
for c1, c2 in itertools.combinations(feature_col,2):
    new_c = f'{c1}_{c2}_divide'
    df[new_c] = df[c1] / df[c2]
    divide_feature.append(new_c)

df.columns.shape

(325,)

In [65]:
# zip 함수를 이용한 row별, 시리즈 그룹별 max, min, max-min, std, sum 을 feature로 생성
# 그 외에도 mean, skew 등을 사용해 볼 수도 있긴 함 

for prefix, g in zip(['one','dered','airmass'], [oneChar_col, dered_col, airmass_col]):
    
    df[f'{prefix}_max'] = df[g].max(axis=1)
    
    df[f'{prefix}_min'] = df[g].min(axis=1)
    
    df[f'{prefix}_diff'] = df[f'{prefix}_max'] - df[f'{prefix}_min']
    
    df[f'{prefix}_std'] = df[g].std(axis=1)
    
    df[f'{prefix}_sum'] = df[g].sum(axis=1)
    
    

In [66]:
# 각 그룹별 max-max, min-min, sum-sum, std-std을 feature로 생성

for c in itertools.combinations(['one','dered','airmass'],2):
    df[f'{c[0]}_{c[1]}_max_diff'] = df[f'{c[0]}_max'] - df[f'{c[1]}_max']
    
    df[f'{c[0]}_{c[1]}_min_diff'] = df[f'{c[0]}_min'] - df[f'{c[1]}_min']
    
    df[f'{c[0]}_{c[1]}_sum_diff'] = df[f'{c[0]}_sum'] - df[f'{c[1]}_sum']
    
    df[f'{c[0]}_{c[1]}_std_diff'] = df[f'{c[0]}_std'] - df[f'{c[1]}_std']
    
    
for c1 in [x for x in df.columns if x != 'class']:
    new_c = f'{c1}_square'
    df[new_c] = df[c1]**2
#    divide_feature.append(new_c)
    


In [67]:
df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,u_g_diff,u_r_diff,u_i_diff,u_z_diff,u_redshift_diff,u_dered_u_diff,u_dered_g_diff,u_dered_r_diff,u_dered_i_diff,u_dered_z_diff,u_nObserve_diff,u_nDetect_diff,u_airmass_u_diff,u_airmass_g_diff,u_airmass_r_diff,u_airmass_i_diff,u_airmass_z_diff,g_r_diff,g_i_diff,g_z_diff,g_redshift_diff,g_dered_u_diff,g_dered_g_diff,g_dered_r_diff,g_dered_i_diff,g_dered_z_diff,g_nObserve_diff,g_nDetect_diff,g_airmass_u_diff,g_airmass_g_diff,g_airmass_r_diff,...,dered_z_airmass_i_divide_square,dered_z_airmass_z_divide_square,nObserve_nDetect_divide_square,nObserve_airmass_u_divide_square,nObserve_airmass_g_divide_square,nObserve_airmass_r_divide_square,nObserve_airmass_i_divide_square,nObserve_airmass_z_divide_square,nDetect_airmass_u_divide_square,nDetect_airmass_g_divide_square,nDetect_airmass_r_divide_square,nDetect_airmass_i_divide_square,nDetect_airmass_z_divide_square,airmass_u_airmass_g_divide_square,airmass_u_airmass_r_divide_square,airmass_u_airmass_i_divide_square,airmass_u_airmass_z_divide_square,airmass_g_airmass_r_divide_square,airmass_g_airmass_i_divide_square,airmass_g_airmass_z_divide_square,airmass_r_airmass_i_divide_square,airmass_r_airmass_z_divide_square,airmass_i_airmass_z_divide_square,one_max_square,one_min_square,one_diff_square,one_std_square,one_sum_square,dered_max_square,dered_min_square,dered_diff_square,dered_std_square,dered_sum_square,airmass_max_square,airmass_min_square,airmass_diff_square,airmass_std_square,airmass_sum_square,one_dered_max_diff_square,one_dered_min_diff_square,one_dered_sum_diff_square,one_dered_std_diff_square,one_airmass_max_diff_square,one_airmass_min_diff_square,one_airmass_sum_diff_square,one_airmass_std_diff_square,dered_airmass_max_diff_square,dered_airmass_min_diff_square,dered_airmass_sum_diff_square,dered_airmass_std_diff_square
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,2.9272,4.2545,5.5915,6.3243,23.2640,0.1397,3.0062,4.3088,5.6318,6.3550,5.2640,5.2640,22.0742,22.0733,22.0750,22.0746,22.0738,1.3273,2.6643,3.3972,20.3369,-2.7875,0.0790,1.3817,2.7047,3.4278,2.3368,2.3368,19.1470,19.1461,19.1478,...,202.1202,201.8313,1.0000,228.8882,228.5358,229.1905,229.0456,228.7182,228.8882,228.5358,229.1905,229.0456,228.7182,0.9985,1.0013,1.0007,0.9993,1.0029,1.0022,1.0008,0.9994,0.9979,0.9986,541.2117,286.9503,39.9974,6.2471,9452.1710,534.7314,285.9123,38.6302,6.0329,9385.3895,1.4177,1.4137,2.8968e-06,4.5358e-07,35.3904,0.0195,0.0009,0.1184,0.0019,487.2295,248.0823,8330.8137,6.2437,481.0819,247.1172,8268.1258,6.0296
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,0.9902,1.5997,1.7837,1.8832,15.0476,0.0857,1.0476,1.6408,1.8159,1.9175,14.0521,14.0521,13.7989,13.7943,13.8034,13.8012,13.7966,0.6096,0.7935,0.8930,14.0575,-0.9045,0.0574,0.6506,0.8257,0.9273,13.0620,13.0620,12.8087,12.8041,12.8132,...,110.2368,109.4417,1.0000,0.6367,0.6321,0.6413,0.6390,0.6344,0.6367,0.6321,0.6413,0.6390,0.6344,0.9927,1.0072,1.0036,0.9964,1.0146,1.0110,1.0037,0.9964,0.9892,0.9928,226.5671,173.4210,3.5465,0.6095,4761.5415,223.9937,172.5198,3.3553,0.5754,4727.0108,1.5822,1.5594,8.2337e-05,1.2865e-05,39.2675,0.0073,0.0012,0.0628,0.0005,190.2830,142.0907,3935.9992,0.6039,187.9253,141.2750,3904.6099,0.5700
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,0.9610,1.2501,1.3928,1.4364,16.7859,0.1787,1.0998,1.3463,1.4647,1.4903,14.7864,14.7864,15.7639,15.7623,15.7654,15.7647,15.7631,0.2891,0.4319,0.4754,15.8250,-0.7822,0.1388,0.3854,0.5037,0.5293,13.8254,13.8254,14.8029,14.8013,14.8044,...,224.1238,223.4405,1.0000,3.8259,3.8139,3.83

## Adding PCA 

In [68]:
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import KernelPCA

In [69]:
#default pca - 
temp = df.columns
temp = temp.drop('class')
pca = PCA(n_components=0.90)# 주성분 개수 설정 
X_pca = pca.fit_transform(df[temp])
print('eigen_value :', pca.explained_variance_)
print('explained variance ratio :', pca.explained_variance_ratio_)

X_pca_t = X_pca.T

for i , ele in enumerate(X_pca_t):
    PCAn = 'PCA'+str(i)
    df[PCAn] = ele
df

eigen_value : [6.35796473e+36]
explained variance ratio : [0.99985116]


,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,u_g_diff,u_r_diff,u_i_diff,u_z_diff,u_redshift_diff,u_dered_u_diff,u_dered_g_diff,u_dered_r_diff,u_dered_i_diff,u_dered_z_diff,u_nObserve_diff,u_nDetect_diff,u_airmass_u_diff,u_airmass_g_diff,u_airmass_r_diff,u_airmass_i_diff,u_airmass_z_diff,g_r_diff,g_i_diff,g_z_diff,g_redshift_diff,g_dered_u_diff,g_dered_g_diff,g_dered_r_diff,g_dered_i_diff,g_dered_z_diff,g_nObserve_diff,g_nDetect_diff,g_airmass_u_diff,g_airmass_g_diff,g_airmass_r_diff,...,dered_z_airmass_z_divide_square,nObserve_nDetect_divide_square,nObserve_airmass_u_divide_square,nObserve_airmass_g_divide_square,nObserve_airmass_r_divide_square,nObserve_airmass_i_divide_square,nObserve_airmass_z_divide_square,nDetect_airmass_u_divide_square,nDetect_airmass_g_divide_square,nDetect_airmass_r_divide_square,nDetect_airmass_i_divide_square,nDetect_airmass_z_divide_square,airmass_u_airmass_g_divide_square,airmass_u_airmass_r_divide_square,airmass_u_airmass_i_divide_square,airmass_u_airmass_z_divide_square,airmass_g_airmass_r_divide_square,airmass_g_airmass_i_divide_square,airmass_g_airmass_z_divide_square,airmass_r_airmass_i_divide_square,airmass_r_airmass_z_divide_square,airmass_i_airmass_z_divide_square,one_max_square,one_min_square,one_diff_square,one_std_square,one_sum_square,dered_max_square,dered_min_square,dered_diff_square,dered_std_square,dered_sum_square,airmass_max_square,airmass_min_square,airmass_diff_square,airmass_std_square,airmass_sum_square,one_dered_max_diff_square,one_dered_min_diff_square,one_dered_sum_diff_square,one_dered_std_diff_square,one_airmass_max_diff_square,one_airmass_min_diff_square,one_airmass_sum_diff_square,one_airmass_std_diff_square,dered_airmass_max_diff_square,dered_airmass_min_diff_square,dered_airmass_sum_diff_square,dered_airmass_std_diff_square,PCA0
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,2.9272,4.2545,5.5915,6.3243,23.2640,0.1397,3.0062,4.3088,5.6318,6.3550,5.2640,5.2640,22.0742,22.0733,22.0750,22.0746,22.0738,1.3273,2.6643,3.3972,20.3369,-2.7875,0.0790,1.3817,2.7047,3.4278,2.3368,2.3368,19.1470,19.1461,19.1478,...,201.8313,1.0000,228.8882,228.5358,229.1905,229.0456,228.7182,228.8882,228.5358,229.1905,229.0456,228.7182,0.9985,1.0013,1.0007,0.9993,1.0029,1.0022,1.0008,0.9994,0.9979,0.9986,541.2117,286.9503,39.9974,6.2471,9452.1710,534.7314,285.9123,38.6302,6.0329,9385.3895,1.4177,1.4137,2.8968e-06,4.5358e-07,35.3904,0.0195,0.0009,0.1184,0.0019,487.2295,248.0823,8330.8137,6.2437,481.0819,247.1172,8268.1258,6.0296,-5.0059e+15
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,0.9902,1.5997,1.7837,1.8832,15.0476,0.0857,1.0476,1.6408,1.8159,1.9175,14.0521,14.0521,13.7989,13.7943,13.8034,13.8012,13.7966,0.6096,0.7935,0.8930,14.0575,-0.9045,0.0574,0.6506,0.8257,0.9273,13.0620,13.0620,12.8087,12.8041,12.8132,...,109.4417,1.0000,0.6367,0.6321,0.6413,0.6390,0.6344,0.6367,0.6321,0.6413,0.6390,0.6344,0.9927,1.0072,1.0036,0.9964,1.0146,1.0110,1.0037,0.9964,0.9892,0.9928,226.5671,173.4210,3.5465,0.6095,4761.5415,223.9937,172.5198,3.3553,0.5754,4727.0108,1.5822,1.5594,8.2337e-05,1.2865e-05,39.2675,0.0073,0.0012,0.0628,0.0005,190.2830,142.0907,3935.9992,0.6039,187.9253,141.2750,3904.6099,0.5700,-5.0060e+15
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,0.9610,1.2501,1.3928,1.4364,16.7859,0.1787,1.0998,1.3463,1.4647,1.4903,14.7864,14.7864,15.7639,15.7623,15.7654,15.7647,15.7631,0.2891,0.4319,0.4754,15.8250,-0.7822,0.1388,0.3854,0.5037,0.5293,13.8254,13.8254,14.8029,14.8013,14.8044,...,223.4405,1.0000,3.8259,3.8139,3.8373,3.8317,3.8200,3.8259,3.8139

In [70]:
X_pca.shape

(399978, 1)

In [71]:
# incrementalPCA - difference is just dividing n_batches for memory 
n_batches = 100
inc_pca = IncrementalPCA(n_components=36)#temp.size*0.5)
for batch_x in np.array_split(df[temp], n_batches):
    print(".", end="") # not shown in the book
    inc_pca.partial_fit(batch_x)
    
X_inc = inc_pca.transform(df[temp])


# print('eigen_value :', inc_pca.explained_variance_)
# print('explained variance ratio :', inc_pca.explained_variance_ratio_)

# X_inc

X_inc_t = X_inc.transpose()

....................................................................................................

In [72]:
for i , ele in enumerate(X_inc_t):
    incPCAn = 'incPCA'+str(i)
    df[incPCAn] = ele
df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,u_g_diff,u_r_diff,u_i_diff,u_z_diff,u_redshift_diff,u_dered_u_diff,u_dered_g_diff,u_dered_r_diff,u_dered_i_diff,u_dered_z_diff,u_nObserve_diff,u_nDetect_diff,u_airmass_u_diff,u_airmass_g_diff,u_airmass_r_diff,u_airmass_i_diff,u_airmass_z_diff,g_r_diff,g_i_diff,g_z_diff,g_redshift_diff,g_dered_u_diff,g_dered_g_diff,g_dered_r_diff,g_dered_i_diff,g_dered_z_diff,g_nObserve_diff,g_nDetect_diff,g_airmass_u_diff,g_airmass_g_diff,g_airmass_r_diff,...,airmass_sum_square,one_dered_max_diff_square,one_dered_min_diff_square,one_dered_sum_diff_square,one_dered_std_diff_square,one_airmass_max_diff_square,one_airmass_min_diff_square,one_airmass_sum_diff_square,one_airmass_std_diff_square,dered_airmass_max_diff_square,dered_airmass_min_diff_square,dered_airmass_sum_diff_square,dered_airmass_std_diff_square,PCA0,incPCA0,incPCA1,incPCA2,incPCA3,incPCA4,incPCA5,incPCA6,incPCA7,incPCA8,incPCA9,incPCA10,incPCA11,incPCA12,incPCA13,incPCA14,incPCA15,incPCA16,incPCA17,incPCA18,incPCA19,incPCA20,incPCA21,incPCA22,incPCA23,incPCA24,incPCA25,incPCA26,incPCA27,incPCA28,incPCA29,incPCA30,incPCA31,incPCA32,incPCA33,incPCA34,incPCA35
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,2.9272,4.2545,5.5915,6.3243,23.2640,0.1397,3.0062,4.3088,5.6318,6.3550,5.2640,5.2640,22.0742,22.0733,22.0750,22.0746,22.0738,1.3273,2.6643,3.3972,20.3369,-2.7875,0.0790,1.3817,2.7047,3.4278,2.3368,2.3368,19.1470,19.1461,19.1478,...,35.3904,0.0195,0.0009,0.1184,0.0019,487.2295,248.0823,8330.8137,6.2437,481.0819,247.1172,8268.1258,6.0296,-5.0059e+15,-5.0059e+15,-8.7600e+13,-6.1014e+12,-1.7965e+12,7.1087e+10,-5.3767e+05,-1326.9641,-23381.5230,4712.6074,-2183.8273,-1700.8342,-54.9083,222.7140,-11.0819,-5.0446,606.5891,-533.5235,489.0111,1011.4944,546.7057,341.9102,16.5058,376.7350,334.1260,840.5245,574.0601,312.6359,1373.1673,-356.1477,-293.7130,275.2114,221.4889,57.5377,-363.0976,100.7519,-527.2350
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,0.9902,1.5997,1.7837,1.8832,15.0476,0.0857,1.0476,1.6408,1.8159,1.9175,14.0521,14.0521,13.7989,13.7943,13.8034,13.8012,13.7966,0.6096,0.7935,0.8930,14.0575,-0.9045,0.0574,0.6506,0.8257,0.9273,13.0620,13.0620,12.8087,12.8041,12.8132,...,39.2675,0.0073,0.0012,0.0628,0.0005,190.2830,142.0907,3935.9992,0.6039,187.9253,141.2750,3904.6099,0.5700,-5.0060e+15,-5.0060e+15,-8.7625e+13,-6.1031e+12,-1.7966e+12,7.0013e+10,8.3351e+03,-3431.8977,1928.8752,156.8968,147.7179,200.8606,-73.6154,-536.2911,-66.1396,-223.5163,-1954.7136,1014.6221,-1047.4438,-1893.4088,-1898.7184,-927.1959,37.6836,-521.7631,-765.1691,-1782.8078,-736.1339,-1277.9029,-3182.4104,70.1215,833.7306,-399.0376,-687.7840,-197.9622,1786.2652,-1120.9165,618.9979
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,0.9610,1.2501,1.3928,1.4364,16.7859,0.1787,1.0998,1.3463,1.4647,1.4903,14.7864,14.7864,15.7639,15.7623,15.7654,15.7647,15.7631,0.2891,0.4319,0.4754,15.8250,-0.7822,0.1388,0.3854,0.5037,0.5293,13.8254,13.8254,14.8029,14.8013,14.8044,...,26.1388,0.0319,0.0029,0.2912,0.0022,248.4495,205.3216,5443.3537,0.3507,242.8469,203.7784,5364.0217,0.2970,-5.0060e+15,-5.0060e+15,-8.7625e+13,-6.1031e+12,-1.7966e+12,7.0016e+10,7.6226e+04,-2674.2909,-566.4193,-329.6157,237.2951,210.6562,-24.0446,-282.0743,-8.7146,-100.7589,-1141.8465,615.9489,-561.1846,-1045.2579,-1068.9911,-369.9672,-49.7325,-334.5320,-460.7573,-876.1910,-409.7961,-826.7397,-1732.6720,-9.4748,472.4516,-201.8199,-353.5175,-110.7676,968.5450,-717.7584,415.8583
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19

# 훈련셋과 테스트셋 나누기

In [73]:
trn2 = df.iloc[:319978]
tst2 = df.iloc[319978:]
tst2.drop('class', axis =1 , inplace = True)
trn2.shape, tst2.shape

((319978, 740), (80000, 739))

# outlier

In [74]:
# 전체 데이터가 아닌 각 클래스별(0,1,2)로 이상치 처리
# Train_copy1 = Train_set.copy()

def remove_outlier(data, column):
    
    print(column)
    # 클래스 0처리
    column_data_0 = data[data['class']==0][column]
    quan_25_0 = np.percentile(column_data_0.values, 0.1) # 1분위수
    quan_75_0 = np.percentile(column_data_0.values, 99.9) # 3분위수
    iqr_0 = quan_75_0 - quan_25_0 # IQR 계산
    iqr_0 *= 2
    lowest_0 = quan_25_0 - iqr_0 # 최솟값 설정
    highest_0 = quan_75_0 + iqr_0 # 최댓값 설정
    outlier_index_0 = column_data_0[(column_data_0 < lowest_0) | (column_data_0 > highest_0)].index # 최솟값 ~ 최댓값 사이의 값만 출력
    print(len(outlier_index_0))
    
    # 클래스 1처리
    column_data_1 = data[data['class']==1][column]
    quan_25_1 = np.percentile(column_data_1.values, 0.1) # 1분위수
    quan_75_1 = np.percentile(column_data_1.values, 99.9) # 3분위수
    iqr_1 = quan_75_1 - quan_25_1 # IQR 계산
    iqr_1 *= 2
    lowest_1 = quan_25_1 - iqr_1 # 최솟값 설정
    highest_1 = quan_75_1 + iqr_1 # 최댓값 설정
    outlier_index_1 = column_data_1[(column_data_1 < lowest_1 )| (column_data_1 > highest_1)].index # 최솟값 ~ 최댓값 사이의 값만 출력
    print(len(outlier_index_1))
    
    # 클래스 2 처리
    column_data_2 = data[data['class']==2][column]
    quan_25_2 = np.percentile(column_data_2.values, 0.1) # 1분위수
    quan_75_2 = np.percentile(column_data_2.values, 99.9) # 3분위수
    iqr_2 = quan_75_2 - quan_25_2 # IQR 계산
    iqr_2 *= 2
    lowest_2 = quan_25_2 - iqr_2 # 최솟값 설정
    highest_2 = quan_75_2 + iqr_2 # 최댓값 설정
    outlier_index_2 = column_data_2[(column_data_2 < lowest_2 )| (column_data_2 > highest_2)].index # 최솟값 ~ 최댓값 사이의 값만 출력
    print(len(outlier_index_2))
    # outlier 행 제거
    data.drop(list(set(list(outlier_index_0) + list(outlier_index_1) + list(outlier_index_2))), axis = 0, inplace = True)
    print(data.shape)
    return data

In [75]:
col = list(trn2.columns)
col.remove('redshift')
col.remove('class')
# col

In [76]:
df2 = trn2.copy()
for i in col:
    
    ROT = remove_outlier(df2, i)
    
    df2 = ROT

df2

u
3
0
0
(319975, 740)
g
0
0
1
(319974, 740)
r
0
0
1
(319973, 740)
i
1
0
0
(319972, 740)
z
0
0
1
(319971, 740)
dered_u
0
0
0
(319971, 740)
dered_g
0
0
0
(319971, 740)
dered_r
0
0
0
(319971, 740)
dered_i
0
0
0
(319971, 740)
dered_z
0
0
0
(319971, 740)
nObserve
0
0
0
(319971, 740)
nDetect
0
0
0
(319971, 740)
airmass_u
0
0
0
(319971, 740)
airmass_g
0
0
0
(319971, 740)
airmass_r
0
0
0
(319971, 740)
airmass_i
0
0
0
(319971, 740)
airmass_z
0
0
0
(319971, 740)
u_g_diff
2
3
2
(319964, 740)
u_r_diff
0
0
1
(319963, 740)
u_i_diff
0
1
0
(319962, 740)
u_z_diff
1
1
1
(319959, 740)
u_redshift_diff
15
0
0
(319944, 740)
u_dered_u_diff
0
2
10
(319932, 740)
u_dered_g_diff
0
0
0
(319932, 740)
u_dered_r_diff
0
0
0
(319932, 740)
u_dered_i_diff
0
0
0
(319932, 740)
u_dered_z_diff
0
0
0
(319932, 740)
u_nObserve_diff
0
0
0
(319932, 740)
u_nDetect_diff
0
0
0
(319932, 740)
u_airmass_u_diff
0
0
0
(319932, 740)
u_airmass_g_diff
0
0
0
(319932, 740)
u_airmass_r_diff
0
0
0
(319932, 740)
u_airmass_i_diff
0
0
0
(319932, 

0
0
0
(319401, 740)
i_redshift_divide
2
0
0
(319399, 740)
i_dered_u_divide
0
0
0
(319399, 740)
i_dered_g_divide
0
0
0
(319399, 740)
i_dered_r_divide
0
2
5
(319392, 740)
i_dered_i_divide
0
1
0
(319391, 740)
i_dered_z_divide
0
0
0
(319391, 740)
i_nObserve_divide
0
0
0
(319391, 740)
i_nDetect_divide
0
0
0
(319391, 740)
i_airmass_u_divide
0
0
0
(319391, 740)
i_airmass_g_divide
0
0
0
(319391, 740)
i_airmass_r_divide
0
0
0
(319391, 740)
i_airmass_i_divide
0
0
0
(319391, 740)
i_airmass_z_divide
0
0
0
(319391, 740)
z_redshift_divide
1
0
0
(319390, 740)
z_dered_u_divide
0
0
0
(319390, 740)
z_dered_g_divide
0
0
1
(319389, 740)
z_dered_r_divide
0
0
4
(319385, 740)
z_dered_i_divide
0
0
2
(319383, 740)
z_dered_z_divide
0
0
0
(319383, 740)
z_nObserve_divide
0
0
0
(319383, 740)
z_nDetect_divide
0
0
0
(319383, 740)
z_airmass_u_divide
0
0
0
(319383, 740)
z_airmass_g_divide
0
0
0
(319383, 740)
z_airmass_r_divide
0
0
0
(319383, 740)
z_airmass_i_divide
0
0
0
(319383, 740)
z_airmass_z_divide
0
0
0
(319383,

1
2
1
(318650, 740)
i_dered_r_diff_square
3
3
27
(318617, 740)
i_dered_i_diff_square
4
4
6
(318603, 740)
i_dered_z_diff_square
1
3
23
(318576, 740)
i_nObserve_diff_square
0
0
0
(318576, 740)
i_nDetect_diff_square
0
0
0
(318576, 740)
i_airmass_u_diff_square
0
0
0
(318576, 740)
i_airmass_g_diff_square
0
0
0
(318576, 740)
i_airmass_r_diff_square
0
0
0
(318576, 740)
i_airmass_i_diff_square
0
0
0
(318576, 740)
i_airmass_z_diff_square
0
0
0
(318576, 740)
z_redshift_diff_square
0
0
0
(318576, 740)
z_dered_u_diff_square
0
0
0
(318576, 740)
z_dered_g_diff_square
1
1
0
(318574, 740)
z_dered_r_diff_square
1
0
6
(318567, 740)
z_dered_i_diff_square
2
3
18
(318544, 740)
z_dered_z_diff_square
22
7
32
(318483, 740)
z_nObserve_diff_square
0
0
0
(318483, 740)
z_nDetect_diff_square
0
0
0
(318483, 740)
z_airmass_u_diff_square
0
0
0
(318483, 740)
z_airmass_g_diff_square
0
0
0
(318483, 740)
z_airmass_r_diff_square
0
0
0
(318483, 740)
z_airmass_i_diff_square
0
0
0
(318483, 740)
z_airmass_z_diff_square
0
0
0


0
(318212, 740)
redshift_dered_u_divide_square
18
0
0
(318194, 740)
redshift_dered_g_divide_square
9
0
0
(318185, 740)
redshift_dered_r_divide_square
4
0
0
(318181, 740)
redshift_dered_i_divide_square
1
0
0
(318180, 740)
redshift_dered_z_divide_square
0
0
0
(318180, 740)
redshift_nObserve_divide_square
29
0
0
(318151, 740)
redshift_nDetect_divide_square
18
0
0
(318133, 740)
redshift_airmass_u_divide_square
18
0
0
(318115, 740)
redshift_airmass_g_divide_square
12
0
0
(318103, 740)
redshift_airmass_r_divide_square
8
0
0
(318095, 740)
redshift_airmass_i_divide_square
3
0
0
(318092, 740)
redshift_airmass_z_divide_square
3
0
0
(318089, 740)
dered_u_dered_g_divide_square
0
0
0
(318089, 740)
dered_u_dered_r_divide_square
0
0
0
(318089, 740)
dered_u_dered_i_divide_square
0
0
0
(318089, 740)
dered_u_dered_z_divide_square
0
0
0
(318089, 740)
dered_u_nObserve_divide_square
0
0
0
(318089, 740)
dered_u_nDetect_divide_square
0
0
0
(318089, 740)
dered_u_airmass_u_divide_square
0
0
0
(318089, 740)
der

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,u_g_diff,u_r_diff,u_i_diff,u_z_diff,u_redshift_diff,u_dered_u_diff,u_dered_g_diff,u_dered_r_diff,u_dered_i_diff,u_dered_z_diff,u_nObserve_diff,u_nDetect_diff,u_airmass_u_diff,u_airmass_g_diff,u_airmass_r_diff,u_airmass_i_diff,u_airmass_z_diff,g_r_diff,g_i_diff,g_z_diff,g_redshift_diff,g_dered_u_diff,g_dered_g_diff,g_dered_r_diff,g_dered_i_diff,g_dered_z_diff,g_nObserve_diff,g_nDetect_diff,g_airmass_u_diff,g_airmass_g_diff,g_airmass_r_diff,...,airmass_sum_square,one_dered_max_diff_square,one_dered_min_diff_square,one_dered_sum_diff_square,one_dered_std_diff_square,one_airmass_max_diff_square,one_airmass_min_diff_square,one_airmass_sum_diff_square,one_airmass_std_diff_square,dered_airmass_max_diff_square,dered_airmass_min_diff_square,dered_airmass_sum_diff_square,dered_airmass_std_diff_square,PCA0,incPCA0,incPCA1,incPCA2,incPCA3,incPCA4,incPCA5,incPCA6,incPCA7,incPCA8,incPCA9,incPCA10,incPCA11,incPCA12,incPCA13,incPCA14,incPCA15,incPCA16,incPCA17,incPCA18,incPCA19,incPCA20,incPCA21,incPCA22,incPCA23,incPCA24,incPCA25,incPCA26,incPCA27,incPCA28,incPCA29,incPCA30,incPCA31,incPCA32,incPCA33,incPCA34,incPCA35
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,2.9272,4.2545,5.5915,6.3243,23.2640,0.1397,3.0062,4.3088,5.6318,6.3550,5.2640,5.2640,22.0742,22.0733,22.0750,22.0746,22.0738,1.3273,2.6643,3.3972,20.3369,-2.7875,0.0790,1.3817,2.7047,3.4278,2.3368,2.3368,19.1470,19.1461,19.1478,...,35.3904,0.0195,9.4045e-04,0.1184,0.0019,487.2295,248.0823,8330.8137,6.2437,481.0819,247.1172,8268.1258,6.0296,-5.0059e+15,-5.0059e+15,-8.7600e+13,-6.1014e+12,-1.7965e+12,7.1087e+10,-5.3767e+05,-1326.9641,-23381.5230,4712.6074,-2183.8273,-1700.8342,-54.9083,222.7140,-11.0819,-5.0446,606.5891,-533.5235,489.0111,1011.4944,546.7057,341.9102,16.5058,376.7350,334.1260,840.5245,574.0601,312.6359,1373.1673,-356.1477,-293.7130,275.2114,221.4889,57.5377,-363.0976,100.7519,-527.2350
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,0.9902,1.5997,1.7837,1.8832,15.0476,0.0857,1.0476,1.6408,1.8159,1.9175,14.0521,14.0521,13.7989,13.7943,13.8034,13.8012,13.7966,0.6096,0.7935,0.8930,14.0575,-0.9045,0.0574,0.6506,0.8257,0.9273,13.0620,13.0620,12.8087,12.8041,12.8132,...,39.2675,0.0073,1.1738e-03,0.0628,0.0005,190.2830,142.0907,3935.9992,0.6039,187.9253,141.2750,3904.6099,0.5700,-5.0060e+15,-5.0060e+15,-8.7625e+13,-6.1031e+12,-1.7966e+12,7.0013e+10,8.3351e+03,-3431.8977,1928.8752,156.8968,147.7179,200.8606,-73.6154,-536.2911,-66.1396,-223.5163,-1954.7136,1014.6221,-1047.4438,-1893.4088,-1898.7184,-927.1959,37.6836,-521.7631,-765.1691,-1782.8078,-736.1339,-1277.9029,-3182.4104,70.1215,833.7306,-399.0376,-687.7840,-197.9622,1786.2652,-1120.9165,618.9979
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,0.9610,1.2501,1.3928,1.4364,16.7859,0.1787,1.0998,1.3463,1.4647,1.4903,14.7864,14.7864,15.7639,15.7623,15.7654,15.7647,15.7631,0.2891,0.4319,0.4754,15.8250,-0.7822,0.1388,0.3854,0.5037,0.5293,13.8254,13.8254,14.8029,14.8013,14.8044,...,26.1388,0.0319,2.9108e-03,0.2912,0.0022,248.4495,205.3216,5443.3537,0.3507,242.8469,203.7784,5364.0217,0.2970,-5.0060e+15,-5.0060e+15,-8.7625e+13,-6.1031e+12,-1.7966e+12,7.0016e+10,7.6226e+04,-2674.2909,-566.4193,-329.6157,237.2951,210.6562,-24.0446,-282.0743,-8.7146,-100.7589,-1141.8465,615.9489,-561.1846,-1045.2579,-1068.9911,-369.9672,-49.7325,-334.5320,-460.7573,-876.1910,-409.7961,-826.7397,-1732.6720,-9.4748,472.4516,-201.8199,-353.5175,-110.7676,968.5450,-717.7584,415.8583
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.087

In [77]:
target_col = 'class'
feature_col = [x for x in df.columns if df[x].dtype in [np.int64, np.float64] and x !=target_col]
print(feature_col)
pred_col='pred'


['u', 'g', 'r', 'i', 'z', 'redshift', 'dered_u', 'dered_g', 'dered_r', 'dered_i', 'dered_z', 'nObserve', 'nDetect', 'airmass_u', 'airmass_g', 'airmass_r', 'airmass_i', 'airmass_z', 'u_g_diff', 'u_r_diff', 'u_i_diff', 'u_z_diff', 'u_redshift_diff', 'u_dered_u_diff', 'u_dered_g_diff', 'u_dered_r_diff', 'u_dered_i_diff', 'u_dered_z_diff', 'u_nObserve_diff', 'u_nDetect_diff', 'u_airmass_u_diff', 'u_airmass_g_diff', 'u_airmass_r_diff', 'u_airmass_i_diff', 'u_airmass_z_diff', 'g_r_diff', 'g_i_diff', 'g_z_diff', 'g_redshift_diff', 'g_dered_u_diff', 'g_dered_g_diff', 'g_dered_r_diff', 'g_dered_i_diff', 'g_dered_z_diff', 'g_nObserve_diff', 'g_nDetect_diff', 'g_airmass_u_diff', 'g_airmass_g_diff', 'g_airmass_r_diff', 'g_airmass_i_diff', 'g_airmass_z_diff', 'r_i_diff', 'r_z_diff', 'r_redshift_diff', 'r_dered_u_diff', 'r_dered_g_diff', 'r_dered_r_diff', 'r_dered_i_diff', 'r_dered_z_diff', 'r_nObserve_diff', 'r_nDetect_diff', 'r_airmass_u_diff', 'r_airmass_g_diff', 'r_airmass_r_diff', 'r_airmass_i_

In [78]:
from sklearn.feature_selection import SelectFromModel
#LGBM 모델을 기반으로 속성들 선택
select = SelectFromModel(LGBMClassifier(max_depth=27, learning_rate = 0.005, n_estimators = 3000, boosting_type='gbrt',
                                       ))#, threshold = 0.3)
select.fit(df2[feature_col], df2[target_col])

train_sel = select.transform(df2[feature_col])

test_sel = select.transform(tst2[feature_col])

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


In [79]:
# SelectFromModel?

## 모델링

In [80]:
df2.shape

(317661, 740)

In [81]:
train_sel.shape

(317661, 142)

In [82]:
y = df2[target_col]
df2.drop(target_col, axis=1, inplace=True)

feature_name = df2.columns.tolist()
print(y.shape, df2.shape, tst2.shape)

(317661,) (317661, 739) (80000, 739)


In [ ]:
X_trn, X_val, y_trn, y_val = train_test_split(train_sel, y, test_size=.01, random_state=seed)
print(X_trn.shape, X_val.shape, y_trn.shape, y_val.shape)

clf = lgb.LGBMClassifier(objective='multiclass',
                         n_estimators=15000,
                         num_leaves=256,
                         learning_rate=0.003,
                         min_child_samples=10,
                         subsample=.5,
                         subsample_freq=1,
                         colsample_bytree=.8,
                         random_state=seed,
                         max_depth = 27,
                         n_jobs=-1,
                         boosting_type = 'dart')
clf.fit(X_trn, y_trn,
        eval_set=[(X_val, y_val)],
        eval_metric='multiclass',
        early_stopping_rounds=15)

p_val = clf.predict(X_val)


print(f'{accuracy_score(y_val, p_val) * 100:.4f}%')

(314484, 142) (3177, 142) (314484,) (3177,)
[1]	valid_0's multi_logloss: 0.969138
[2]	valid_0's multi_logloss: 0.964507
[3]	valid_0's multi_logloss: 0.95989
[4]	valid_0's multi_logloss: 0.955342
[5]	valid_0's multi_logloss: 0.950787
[6]	valid_0's multi_logloss: 0.946305
[7]	valid_0's multi_logloss: 0.946351
[8]	valid_0's multi_logloss: 0.941904
[9]	valid_0's multi_logloss: 0.937501
[10]	valid_0's multi_logloss: 0.93758
[11]	valid_0's multi_logloss: 0.933188
[12]	valid_0's multi_logloss: 0.93395
[13]	valid_0's multi_logloss: 0.929617
[14]	valid_0's multi_logloss: 0.9253
[15]	valid_0's multi_logloss: 0.921017
[16]	valid_0's multi_logloss: 0.916753
[17]	valid_0's multi_logloss: 0.918204
[18]	valid_0's multi_logloss: 0.913975
[19]	valid_0's multi_logloss: 0.914011
[20]	valid_0's multi_logloss: 0.914252
[21]	valid_0's multi_logloss: 0.915742
[22]	valid_0's multi_logloss: 0.924164
[23]	valid_0's multi_logloss: 0.920024
[24]	valid_0's multi_logloss: 0.932525
[25]	valid_0's multi_logloss: 0.93

[209]	valid_0's multi_logloss: 0.841632
[210]	valid_0's multi_logloss: 0.838197
[211]	valid_0's multi_logloss: 0.83479
[212]	valid_0's multi_logloss: 0.83141
[213]	valid_0's multi_logloss: 0.828034
[214]	valid_0's multi_logloss: 0.831292
[215]	valid_0's multi_logloss: 0.827934
[216]	valid_0's multi_logloss: 0.824602
[217]	valid_0's multi_logloss: 0.825921
[218]	valid_0's multi_logloss: 0.827312
[219]	valid_0's multi_logloss: 0.82398
[220]	valid_0's multi_logloss: 0.825469
[221]	valid_0's multi_logloss: 0.827072
[222]	valid_0's multi_logloss: 0.823739
[223]	valid_0's multi_logloss: 0.820418
[224]	valid_0's multi_logloss: 0.817122
[225]	valid_0's multi_logloss: 0.813856
[226]	valid_0's multi_logloss: 0.810595
[227]	valid_0's multi_logloss: 0.812161
[228]	valid_0's multi_logloss: 0.808924
[229]	valid_0's multi_logloss: 0.805702
[230]	valid_0's multi_logloss: 0.802492
[231]	valid_0's multi_logloss: 0.804343
[232]	valid_0's multi_logloss: 0.801151
[233]	valid_0's multi_logloss: 0.797965
[23

[415]	valid_0's multi_logloss: 0.716227
[416]	valid_0's multi_logloss: 0.713521
[417]	valid_0's multi_logloss: 0.710839
[418]	valid_0's multi_logloss: 0.708177
[419]	valid_0's multi_logloss: 0.705526
[420]	valid_0's multi_logloss: 0.706934
[421]	valid_0's multi_logloss: 0.70831
[422]	valid_0's multi_logloss: 0.70567
[423]	valid_0's multi_logloss: 0.707015
[424]	valid_0's multi_logloss: 0.708416
[425]	valid_0's multi_logloss: 0.709609
[426]	valid_0's multi_logloss: 0.710887
[427]	valid_0's multi_logloss: 0.708227
[428]	valid_0's multi_logloss: 0.710229
[429]	valid_0's multi_logloss: 0.711623
[430]	valid_0's multi_logloss: 0.713012
[431]	valid_0's multi_logloss: 0.710334
[432]	valid_0's multi_logloss: 0.711658
[433]	valid_0's multi_logloss: 0.708997
[434]	valid_0's multi_logloss: 0.710127
[435]	valid_0's multi_logloss: 0.707466
[436]	valid_0's multi_logloss: 0.704817
[437]	valid_0's multi_logloss: 0.706278
[438]	valid_0's multi_logloss: 0.703634
[439]	valid_0's multi_logloss: 0.70494
[44

[621]	valid_0's multi_logloss: 0.601768
[622]	valid_0's multi_logloss: 0.60331
[623]	valid_0's multi_logloss: 0.604372
[624]	valid_0's multi_logloss: 0.602291
[625]	valid_0's multi_logloss: 0.600219
[626]	valid_0's multi_logloss: 0.59816
[627]	valid_0's multi_logloss: 0.598911
[628]	valid_0's multi_logloss: 0.599987
[629]	valid_0's multi_logloss: 0.601418
[630]	valid_0's multi_logloss: 0.599348
[631]	valid_0's multi_logloss: 0.600449
[632]	valid_0's multi_logloss: 0.598372
[633]	valid_0's multi_logloss: 0.599444
[634]	valid_0's multi_logloss: 0.597367
[635]	valid_0's multi_logloss: 0.598764
[636]	valid_0's multi_logloss: 0.5967
[637]	valid_0's multi_logloss: 0.594645
[638]	valid_0's multi_logloss: 0.592607
[639]	valid_0's multi_logloss: 0.590592
[640]	valid_0's multi_logloss: 0.588562
[641]	valid_0's multi_logloss: 0.589671
[642]	valid_0's multi_logloss: 0.591059
[643]	valid_0's multi_logloss: 0.589032
[644]	valid_0's multi_logloss: 0.587025
[645]	valid_0's multi_logloss: 0.587894
[646

[827]	valid_0's multi_logloss: 0.471974
[828]	valid_0's multi_logloss: 0.472746
[829]	valid_0's multi_logloss: 0.47131
[830]	valid_0's multi_logloss: 0.472132
[831]	valid_0's multi_logloss: 0.470702
[832]	valid_0's multi_logloss: 0.47135
[833]	valid_0's multi_logloss: 0.46993
[834]	valid_0's multi_logloss: 0.468519
[835]	valid_0's multi_logloss: 0.46939
[836]	valid_0's multi_logloss: 0.470019
[837]	valid_0's multi_logloss: 0.468604
[838]	valid_0's multi_logloss: 0.469659
[839]	valid_0's multi_logloss: 0.468241
[840]	valid_0's multi_logloss: 0.466822
[841]	valid_0's multi_logloss: 0.465415
[842]	valid_0's multi_logloss: 0.464018
[843]	valid_0's multi_logloss: 0.46467
[844]	valid_0's multi_logloss: 0.463277
[845]	valid_0's multi_logloss: 0.461888
[846]	valid_0's multi_logloss: 0.462639
[847]	valid_0's multi_logloss: 0.46336
[848]	valid_0's multi_logloss: 0.461967
[849]	valid_0's multi_logloss: 0.462909
[850]	valid_0's multi_logloss: 0.463628
[851]	valid_0's multi_logloss: 0.462235
[852]	

...

[8873]	valid_0's multi_logloss: 0.139224
[8874]	valid_0's multi_logloss: 0.139218
[8875]	valid_0's multi_logloss: 0.13922
[8876]	valid_0's multi_logloss: 0.13922
[8877]	valid_0's multi_logloss: 0.139222
[8878]	valid_0's multi_logloss: 0.139214
[8879]	valid_0's multi_logloss: 0.139215
[8880]	valid_0's multi_logloss: 0.139217
[8881]	valid_0's multi_logloss: 0.139218
[8882]	valid_0's multi_logloss: 0.139214
[8883]	valid_0's multi_logloss: 0.139216
[8884]	valid_0's multi_logloss: 0.139224
[8885]	valid_0's multi_logloss: 0.139225
[8886]	valid_0's multi_logloss: 0.139224
[8887]	valid_0's multi_logloss: 0.139226
[8888]	valid_0's multi_logloss: 0.139228
[8889]	valid_0's multi_logloss: 0.139227
[8890]	valid_0's multi_logloss: 0.13922
[8891]	valid_0's multi_logloss: 0.139222
[8892]	valid_0's multi_logloss: 0.139223
[8893]	valid_0's multi_logloss: 0.139215
[8894]	valid_0's multi_logloss: 0.139217
[8895]	valid_0's multi_logloss: 0.139219
[8896]	valid_0's multi_logloss: 0.139218
[8897]	valid_0's mu

[9074]	valid_0's multi_logloss: 0.139153
[9075]	valid_0's multi_logloss: 0.139145
[9076]	valid_0's multi_logloss: 0.139146
[9077]	valid_0's multi_logloss: 0.13915
[9078]	valid_0's multi_logloss: 0.13915
[9079]	valid_0's multi_logloss: 0.13915
[9080]	valid_0's multi_logloss: 0.139152
[9081]	valid_0's multi_logloss: 0.139151
[9082]	valid_0's multi_logloss: 0.139151
[9083]	valid_0's multi_logloss: 0.139156
[9084]	valid_0's multi_logloss: 0.139157
[9085]	valid_0's multi_logloss: 0.139154
[9086]	valid_0's multi_logloss: 0.139155
[9087]	valid_0's multi_logloss: 0.139156
[9088]	valid_0's multi_logloss: 0.139156
[9089]	valid_0's multi_logloss: 0.139157
[9090]	valid_0's multi_logloss: 0.139157
[9091]	valid_0's multi_logloss: 0.139159
[9092]	valid_0's multi_logloss: 0.13916
[9093]	valid_0's multi_logloss: 0.139161
[9094]	valid_0's multi_logloss: 0.139163
[9095]	valid_0's multi_logloss: 0.139164
[9096]	valid_0's multi_logloss: 0.139166
[9097]	valid_0's multi_logloss: 0.139167
[9098]	valid_0's mul

[9275]	valid_0's multi_logloss: 0.139103
[9276]	valid_0's multi_logloss: 0.139104
[9277]	valid_0's multi_logloss: 0.139105
[9278]	valid_0's multi_logloss: 0.139109
[9279]	valid_0's multi_logloss: 0.139109
[9280]	valid_0's multi_logloss: 0.13911
[9281]	valid_0's multi_logloss: 0.139111
[9282]	valid_0's multi_logloss: 0.139112
[9283]	valid_0's multi_logloss: 0.13911
[9284]	valid_0's multi_logloss: 0.13911
[9285]	valid_0's multi_logloss: 0.13911
[9286]	valid_0's multi_logloss: 0.139111
[9287]	valid_0's multi_logloss: 0.139103
[9288]	valid_0's multi_logloss: 0.139103
[9289]	valid_0's multi_logloss: 0.139104
[9290]	valid_0's multi_logloss: 0.139105
[9291]	valid_0's multi_logloss: 0.139105
[9292]	valid_0's multi_logloss: 0.139111
[9293]	valid_0's multi_logloss: 0.139107
[9294]	valid_0's multi_logloss: 0.139107
[9295]	valid_0's multi_logloss: 0.139108
[9296]	valid_0's multi_logloss: 0.139113
[9297]	valid_0's multi_logloss: 0.139114
[9298]	valid_0's multi_logloss: 0.139115
[9299]	valid_0's mul

[9476]	valid_0's multi_logloss: 0.139096
[9477]	valid_0's multi_logloss: 0.139096
[9478]	valid_0's multi_logloss: 0.139089
[9479]	valid_0's multi_logloss: 0.139094
[9480]	valid_0's multi_logloss: 0.139088
[9481]	valid_0's multi_logloss: 0.139089
[9482]	valid_0's multi_logloss: 0.13909
[9483]	valid_0's multi_logloss: 0.139091
[9484]	valid_0's multi_logloss: 0.13909
[9485]	valid_0's multi_logloss: 0.139091
[9486]	valid_0's multi_logloss: 0.139096
[9487]	valid_0's multi_logloss: 0.139097
[9488]	valid_0's multi_logloss: 0.139097
[9489]	valid_0's multi_logloss: 0.139101
[9490]	valid_0's multi_logloss: 0.13911
[9491]	valid_0's multi_logloss: 0.139105
[9492]	valid_0's multi_logloss: 0.139103
[9493]	valid_0's multi_logloss: 0.139103
[9494]	valid_0's multi_logloss: 0.139104
[9495]	valid_0's multi_logloss: 0.139105
[9496]	valid_0's multi_logloss: 0.139105
[9497]	valid_0's multi_logloss: 0.139098
[9498]	valid_0's multi_logloss: 0.139093
[9499]	valid_0's multi_logloss: 0.139094
[9500]	valid_0's mu

[9677]	valid_0's multi_logloss: 0.139103
[9678]	valid_0's multi_logloss: 0.139103
[9679]	valid_0's multi_logloss: 0.139105
[9680]	valid_0's multi_logloss: 0.139104
[9681]	valid_0's multi_logloss: 0.139104
[9682]	valid_0's multi_logloss: 0.139102
[9683]	valid_0's multi_logloss: 0.139101
[9684]	valid_0's multi_logloss: 0.139102
[9685]	valid_0's multi_logloss: 0.139106
[9686]	valid_0's multi_logloss: 0.139113
[9687]	valid_0's multi_logloss: 0.139114
[9688]	valid_0's multi_logloss: 0.139117
[9689]	valid_0's multi_logloss: 0.139117
[9690]	valid_0's multi_logloss: 0.139118
[9691]	valid_0's multi_logloss: 0.139118
[9692]	valid_0's multi_logloss: 0.139114
[9693]	valid_0's multi_logloss: 0.139115
[9694]	valid_0's multi_logloss: 0.139116
[9695]	valid_0's multi_logloss: 0.139111
[9696]	valid_0's multi_logloss: 0.139109
[9697]	valid_0's multi_logloss: 0.139114
[9698]	valid_0's multi_logloss: 0.139114
[9699]	valid_0's multi_logloss: 0.139114
[9700]	valid_0's multi_logloss: 0.139115
[9701]	valid_0's

In [ ]:
print(f'{accuracy_score(y_val, p_val) * 100:.4f}%')
p_tst = clf.predict(test_sel)

In [ ]:
#submission
Submission_set = pd.read_csv(sample_file, index_col=0)

In [ ]:
Submission_set['class'] = p_tst
print(Submission_set.shape)
Submission_set.head()

In [ ]:
#class 값 카운팅
Submission_set[target_col].value_counts()

In [ ]:
#저장하기
Submission_set.to_csv('../sub/lgb_3rd_updated.csv')
